TABU- HEURISTIC 1

In [1]:
import numpy as np
import random
import pandas as pd
import copy
from collections import defaultdict
from networkx import nx
from copy import copy
import time
import pickle
import itertools


In [2]:
adjacency_df=pd.DataFrame(pd.read_csv("C:/Users/Lenovo/Desktop/adjacency_df-n100-m400.csv"))

n = len(adjacency_df)

adjacency_df.columns=(range(1,(n+4)))
adjacency_df=adjacency_df.rename(columns={n+1:'Threshold',n+2:'IsInfected', n+3:'IsRisky' },errors='raise')

adjacency_df.index=(range(1,(n+1)))

m=((adjacency_df.loc[:,range(1,n+1)]>0).sum().sum())/2


zeros=[1,0.3,0.1]
ones=[1,0.5,0.3]

In [3]:
def convert(a):      
    rows, cols = np.where(a.loc[range(1,n+1), range(1,n+1)] > 0)
    edges = zip((rows+1).tolist(), (cols+1).tolist())
    G = nx.Graph()
    G.add_edges_from(edges)

    return G

In [4]:
#parameters
Blocking_Capacity = (int(n/10))*2
step_size=3

In [5]:
#sets

setV = range(1,n+1)
setS = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
setP = adjacency_df.loc[(adjacency_df.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
setRisky= adjacency_df.loc[(adjacency_df.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()


In [6]:
#Preprocessing
preprocessed_matrix = adjacency_df.copy(deep=True)



In [7]:
#2 infected 
preprocessed_matrix.loc[setP,setP] = 0




In [8]:
#Sum of all edges lower than threshold (not infected)
index = preprocessed_matrix.loc[(preprocessed_matrix[setV].sum(axis=1)<((preprocessed_matrix.loc[:, "Threshold"]))) & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)  , range(1,n+1)].index
preprocessed_matrix.loc[index,setV] = 0
preprocessed_matrix.loc[setV,index] = 0



In [9]:
#1 neighbor (not infected not risky)
index2 = preprocessed_matrix.loc[((preprocessed_matrix.loc[setV,setV]>0).sum(axis=1) == 1 & (preprocessed_matrix.loc[:,"IsInfected"]<0.01)) & (preprocessed_matrix.loc[:,"IsRisky"]<0.01), range(1,n+1)].index
preprocessed_matrix.loc[index2,setV] = 0
preprocessed_matrix.loc[setV,index2] = 0

In [10]:
#Percentage assignment
preprocessed_matrix.loc[:,setV]=preprocessed_matrix.loc[:,setV].div(preprocessed_matrix.loc[:,"Threshold"],axis=0)
preprocessed_matrix[preprocessed_matrix[setV] >= 1] = 1


nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [11]:
#!!!!!!!!! Edges below %30 influence are deleted !!!!!!!!!!!
preprocessed_matrix[preprocessed_matrix[setV] <= 0.3] = 0

nonconnected_nodes=preprocessed_matrix.loc[preprocessed_matrix.loc[:,range(1,n+1)].sum(axis=0)==0,:].index
for i in nonconnected_nodes:
    preprocessed_matrix.at[i,i]=1

In [12]:
Graph = convert(preprocessed_matrix)
#Graph.nodes()

In [13]:
infected_neighbors=[]
for node in Graph.nodes():
    if (node) in setP: 
        for neighbors in Graph.neighbors(node):
            infected_neighbors.append(neighbors)

In [14]:
infected_neighbors=np.sort(infected_neighbors).tolist()
infected_neighbors_unique = []

prob_of_selection = []
j=-1
for i in range(len(infected_neighbors)):
    if infected_neighbors[i] not in infected_neighbors_unique:
        infected_neighbors_unique.append(infected_neighbors[i])
        prob_of_selection.append(1)
        j+=1
    else:
        prob_of_selection[j]+=1
        
prob_of_selection=np.array(prob_of_selection)/sum(prob_of_selection)
       

In [15]:

if(Blocking_Capacity<=len(infected_neighbors_unique)):
    initial_solution_neighbor = np.random.choice(infected_neighbors_unique,Blocking_Capacity,replace=False,p=prob_of_selection)
else:
    riskies_potential = []
    for riskies in setRisky:
        if(riskies not in infected_neighbors_unique):
            riskies_potential.append(riskies)
    risky_in_initial = (np.random.choice(riskies_potential,(Blocking_Capacity-len(infected_neighbors_unique)),replace=False)).tolist()
    initial_solution_neighbor = np.array((infected_neighbors_unique + risky_in_initial))
            
            

In [16]:
blocked_nodes = np.zeros(n)
#blocked_nodes[nodes_on_paths[np.argpartition(total_appearance, -Blocking_Capacity)[-Blocking_Capacity:]]-1]=1


In [17]:
blocked_nodes_df=pd.DataFrame(blocked_nodes).T
blocked_nodes_df.columns=(range(1,(n+1)))
blocked_nodes_df
blocked_nodes_df[initial_solution_neighbor]=1

In [18]:
initial_solution_for_tabu=blocked_nodes_df.values.tolist()[0]


In [19]:
#blocked_nodes_df.to_csv('blocked_nodes_df-n'+str(n)+'-m'+str(m)+'.csv',index=False)


In [20]:
def simulate_LTM(removed_nodes,Graph_for_simulation,setP):
    G=Graph_for_simulation.copy()
    #silinen=[]
    for i in range(len(removed_nodes)):
        if(removed_nodes[i]>0):            
            G.remove_node(i+1)
            #silinen.append(i+1)
    converted_list=setP.copy()
    #print('Blocked:'+str(silinen)) 
    
    while(1):
        #start2=time.time()
        converted_list1 = converted_list[:]
        #print('step:'+str(converted_list))
        for node in G.nodes():
            if (node) not in converted_list: 
                total_weight = 0
                for each in G.neighbors(node):
                    if (each) in converted_list:
                        total_weight = total_weight + adjacency_df.at[node,each]
                if total_weight > adjacency_df.at[node,'Threshold']:
                    converted_list.append(node)
        #end2=time.time()
        #print(end2-start2)           
        if set(converted_list1) == set(converted_list):
            break
    objective_value=adjacency_df.loc[converted_list,'IsRisky'].sum()
    #print('FINAL:'+str(converted_list))
    return objective_value 

In [21]:
def NeighborEvaluation(Neighbor,Graph_for_simulation,setP):
    
    ObjVal = simulate_LTM(Neighbor,Graph_for_simulation,setP)
    #ObjVal = (i * Neighbor[i])
    
    return ObjVal

In [22]:
def NeighborhoodSearchAdd(AddDropLimit,IncumbentSolution,TabuAddList,Graph_for_simulation,setP):

    ZeroIndices = []
    OneIndices = []
    zeros_for_comb=[]
    
    for i in range(len(IncumbentSolution)):
        if IncumbentSolution[i] == 0:
            ZeroIndices.append(i)
            if (i+1 not in setP):
                if (i not in TabuAddList):
                    zeros_for_comb.append(i)
        else:
            OneIndices.append(i)
    
    AllNeighbor = []
    AllNeighborObjVals = []
    Moves = []

    #random_zeros= random.sample(range(len(zeros_for_comb)),int((len(zeros_for_comb)*zeros[0])))

    for subset in itertools.combinations(zeros_for_comb, AddDropLimit):
        Moves.append(subset)
        Temp = copy(IncumbentSolution)
        for subset_element in subset:
            Temp[subset_element] = 1  
        #print('Exchanged:'+str(OneIndices[i]+1)+','+str(ZeroIndices[j]+1))
        TempObjVal = NeighborEvaluation(Temp,Graph_for_simulation,setP)+n
        #print('AddObjective: '+str(TempObjVal))
        #print("neighborda çıktım")
        AllNeighbor.append(Temp)
        AllNeighborObjVals.append(TempObjVal)
        #end=time.time()
        #print(end-start)
    BestIndex = AllNeighborObjVals.index(min(AllNeighborObjVals))
    BestSolution = AllNeighbor[BestIndex]
    BestObjVal = AllNeighborObjVals[BestIndex]
    BestMove = Moves[BestIndex]
    #print('AddBEST: '+str(BestObjVal))
    #print('AddBESTSOLNLEN: '+str(sum(BestSolution)))
    return BestSolution, BestObjVal, BestMove

In [23]:
def NeighborhoodSearchDrop(AddDropLimit,IncumbentSolution,TabuDropList,Graph_for_simulation,setP):

    ZeroIndices = []
    OneIndices = []
    ones_for_comb=[]
    
    for i in range(len(IncumbentSolution)):
        if IncumbentSolution[i] == 0:
            ZeroIndices.append(i)
        else:
            OneIndices.append(i)
            if i not in TabuDropList:
                ones_for_comb.append(i)
    #print(len(TabuDropList))
    #print(sum(IncumbentSolution))
    AllNeighbor = []
    AllNeighborObjVals = []
    Moves = []

    #random_ones= random.sample(range(len(ones_for_comb)),int((len(ones_for_comb)*ones[0])))

    for subset in itertools.combinations(ones_for_comb, AddDropLimit):
        Moves.append(subset)
        Temp = copy(IncumbentSolution)
        for subset_element in subset:
            Temp[subset_element] = 0  
        #print('Exchanged:'+str(OneIndices[i]+1)+','+str(ZeroIndices[j]+1))
        TempObjVal = NeighborEvaluation(Temp,Graph_for_simulation,setP)
        #print('DropObjective: '+str(TempObjVal))
        #print("neighborda çıktım")
        AllNeighbor.append(Temp)
        AllNeighborObjVals.append(TempObjVal)
        #end=time.time()
        #print(end-start)
    BestIndex = AllNeighborObjVals.index(min(AllNeighborObjVals))
    BestSolution = AllNeighbor[BestIndex]
    BestObjVal = AllNeighborObjVals[BestIndex]
    BestMove = Moves[BestIndex]
    #print('DropBEST: '+str(BestObjVal))
    #print('DropBESTSOLNLEN: '+str(sum(BestSolution)))
    return BestSolution, BestObjVal, BestMove

In [24]:
def TabuSearchAddDrop(AddDropLimit,NumberOfOnes,SpinningLimit,MaxIter,TabuTenure,Graph_for_simulation,setP,initial_solution_for_tabu):
    
    random.seed(123)
    
    InitialSolution = initial_solution_for_tabu.copy() 

    IncumbentSolution = InitialSolution
    IncumbentObjVal = NeighborEvaluation(IncumbentSolution,Graph_for_simulation,setP)
    #print("tabuda çıktım")
    BestSolution = IncumbentSolution
    BestObjVal = IncumbentObjVal
    Trajectory = [IncumbentObjVal]
    BestTrajectory = [BestObjVal]
    
    TabuAddList=[]
    TabuDropList=[]
    #for i in setS:
     #   for j in setP:
     #       TabuList = [j,i]
    
    SpinningIndex = 0
    Iteration = 0
    start_CPU=time.time()
    CPU_time=0
    
    while ((Iteration < MaxIter) and CPU_time<=7200):
        #start=time.time()
        
        #START WITH ADD!
        if (Iteration%2):#DROP OPERATOR
            #print('__________________')
            #print('DROPA GİRDİM'+str(Iteration))
            #print('__________________')
            Out = NeighborhoodSearchDrop(AddDropLimit,IncumbentSolution,TabuDropList,Graph_for_simulation,setP)
            #end=time.time()
            #print(end-start)
            BestNeighbor = Out[0]
            BestNeighborObjVal = Out[1]
            BestNeighborMove = Out[2]
            
            if BestObjVal <= IncumbentObjVal:

                IncumbentSolution = BestNeighbor
                IncumbentObjVal = BestNeighborObjVal
                Trajectory.append(IncumbentObjVal)
                if IncumbentObjVal < BestObjVal:
                    BestObjVal = IncumbentObjVal
                    BestSolution = IncumbentSolution
                    SpinningIndex = 0
                else:
                    SpinningIndex += 1
                BestTrajectory.append(BestObjVal)
                for size in BestNeighborMove:
                    TabuAddList.append(size)

            else:                
                BestTrajectory.append(BestObjVal)
                SpinningIndex += 1
            
            if len(TabuAddList) > TabuTenure:
                for add in range(AddDropLimit): 
                    TabuAddList.pop(0)
                
            
        else:#ADD OPERATOR
            #print('__________________')
            #print('ADDE GİRDİM'+str(Iteration))
            #print('__________________')
            Out = NeighborhoodSearchAdd(AddDropLimit,IncumbentSolution,TabuAddList,Graph_for_simulation,setP)
            #end=time.time()
            #print(end-start)
            BestNeighbor = Out[0]
            BestNeighborObjVal = Out[1]
            BestNeighborMove = Out[2]
            
            IncumbentSolution = BestNeighbor
            IncumbentObjVal = BestNeighborObjVal
            
            
            Trajectory.append(IncumbentObjVal)
            BestTrajectory.append(BestObjVal)
            for size in BestNeighborMove:
                TabuDropList.append(size)
            
            if len(TabuDropList) > TabuTenure:
                for add in range(AddDropLimit): 
                    TabuDropList.pop(0)
            
        if SpinningIndex > SpinningLimit:
            
            Iteration = MaxIter
            
        else:
        
            Iteration += 1
        end_CPU=time.time()
        CPU_time=end_CPU-start_CPU
    return BestSolution, BestObjVal, BestTrajectory, Trajectory



In [25]:

SpinningLimit = 20
MaxIter = n*5
TabuTenure = 12
Graph2 = convert(adjacency_df)
start1=time.time()
Tabu_Results = TabuSearchAddDrop(2,Blocking_Capacity,SpinningLimit,MaxIter,TabuTenure,Graph2,setP,initial_solution_for_tabu)
end1=time.time()
print(end1-start1)

166.78810095787048


In [26]:
print('TABU ADD-DROP OBJECTIVE VALUE: '+str(Tabu_Results[1]))

TABU ADD-DROP OBJECTIVE VALUE: 10.0


In [27]:
print('TABU ADD-DROP TIME: '+str(end1-start1))

TABU ADD-DROP TIME: 166.78810095787048


In [28]:
print('NEW_CONSTRUCTION OBJECTIVE VALUE: '+str(simulate_LTM(initial_solution_for_tabu,Graph,setP)))

NEW_CONSTRUCTION OBJECTIVE VALUE: 23.0
